# Question 1

Write a script that creates a new polygon feature class containing a single (square) polygon with the following coordinates: (0, 0), (0, 1,000), (1,000, 0), and (1,000, 1,000). (5 pt.)

In [1]:
import arcpy
workspace =  r"C:\Geog503\data\Results"
out_file = "Q1_polygon.shp"
arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True
pnt = arcpy.Point()
array = arcpy.Array()
coordList = [[0,0],[0,1000],[1000,1000], [1000,0]]
for coord in coordList:
    pnt.X = coord[0]
    pnt.Y = coord[1]
    array.add(pnt)
fc = arcpy.Polygon(array)
arcpy.CopyFeatures_management(fc,out_file)

<Result 'C:\\geog503\\data\\Results\\Q1_polygon.shp'>

# Question 2

Write a script that determines the perimeter (in meters) and area (in square meters) of each of the individual islands of the Hawaii.shp feature class. Recall that this is a multipart feature class. (5 pt.)

In [2]:
import arcpy
arcpy.env.workspace = r"C:\Geog503\data"
fc = "hawaii.shp"
g = arcpy.Geometry()
geomList = arcpy.CopyFeatures_management(fc,g)

for geom in geomList:
    print("partCount = {}".format(geom.partCount))
    partCount = geom.partCount
    for i in range(0, partCount):
        part = geom.getPart(i)
        fc = arcpy.Polygon(part)
        length = fc.length
        area = fc.area
        print("Part {0:2}: Length = {1}, Area = {2}".format(str(i),str(length),str(area)))

partCount = 11
Part 0 : Length = 498725.055304, Area = 10486352990.4
Part 1 : Length = 56881.0037528, Area = 119076863.937
Part 2 : Length = 79591.0682892, Area = 364740852.744
Part 3 : Length = 237012.259994, Area = 1907482650.51
Part 4 : Length = 161088.602656, Area = 685386630.295
Part 5 : Length = 4352.76437225, Area = 1031618.8376
Part 6 : Length = 4695.32817565, Area = 1256964.62017
Part 7 : Length = 272519.587261, Area = 1577775022.22
Part 8 : Length = 78651.6156499, Area = 196286563.822
Part 9 : Length = 4013.22238472, Area = 856335.844777
Part 10: Length = 172463.404798, Area = 1449442868.45


# Question 3

Write a script that creates an envelope polygon feature class for the Hawaii.shp feature class. There is actually a tool that accomplishes this called Minimum Bounding Geometry. You can look at the tool to get some ideas, but your script needs to work directly with the geometry properties. (10 pt.)

In [3]:
import arcpy
fc = r"C:\Geog503\data\hawaii.shp"
workspace = r"C:\Geog503\data\Results"
out_file = "Q3_envelope.shp"
arcpy.env.workspace = workspace
geomList = arcpy.CopyFeatures_management(fc,arcpy.Geometry())
pnt = arcpy.Point()
array = arcpy.Array()
for geom in geomList:
    coord_list = geom.hullRectangle.split(" ")
    print(coord_list)
    point_count = len(coord_list)/2
    for i in range(0, point_count):
        pnt.X = coord_list[i*2]
        pnt.Y = coord_list[i*2+1]
        array.add(pnt)
poly = arcpy.Polygon(array)
arcpy.CopyFeatures_management(poly,out_file)        

[u'369452.444658486', u'2413270.8632363', u'453956.568571401', u'2540498.57597544', u'966495.579671934', u'2200072.27127229', u'881991.455755093', u'2072844.55853016']


<Result 'C:\\geog503\\data\\Results\\Q3_envelope.shp'>

# Question 4

Write a script that creates a Polygon shapefile based on the coordinates from the coordinates.txt, which contains coordinate (X, Y) pairs for 11 features. You should use relevant Geometry classes to perform this task. Do NOT use InsertCursor function. Use the spatial reference of Hawaii.shp for the resulting Polygon shapefile. (20 pt.)

In [4]:
import arcpy
import os
folder =  r"C:\Geog503\data"
workspace = os.path.join(folder,"Results")
in_file = os.path.join(folder, "coordinates.txt")
out_file = "polygon.shp"
out_prj = os.path.join(folder, "holes.prj")

arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True

fcList = []
fc = []

f = open(in_file)

for line in f:
    if line.startswith("Feature"):
        if len(fc)>0:  # skip the first line
            fcList.append(fc) 
            fc = []
    else:
        XY = line.split(",")
        X = float(XY[0].split("=")[1])
        Y = float(XY[1].split("=")[1])
        fc.append([X,Y])

fcList.append(fc)  # add the last polygon to the list
pnt = arcpy.Point()
geomList =[]

spatialRef = arcpy.SpatialReference(out_prj)
i = 0
for fc in fcList:
    array = arcpy.Array()
    for point in fc:
        pnt.X = point[0]
        pnt.Y = point[1]
        array.add(pnt)
    fcGeometry = arcpy.Polygon(array,spatialRef)
    geomList.append(fcGeometry)
    print("Feature {:2}: {:5} points".format(str(i),str(array.count)))
    i = i + 1
arcpy.CopyFeatures_management(geomList, out_file)

Feature 0 : 440   points
Feature 1 : 64    points
Feature 2 : 73    points
Feature 3 : 240   points
Feature 4 : 141   points
Feature 5 : 12    points
Feature 6 : 13    points
Feature 7 : 277   points
Feature 8 : 81    points
Feature 9 : 12    points
Feature 10: 193   points


<Result 'C:\\geog503\\data\\Results\\polygon.shp'>